# <span style="color:#1f77b4">**Generative AI 01 - Basic ChatBot**</span>


## <span style="color:#1f77b4">**Configure OpenAI secrets**</span>

Load the Azure OpenAI endpoint, key, API version, and deployment name from the Databricks secret scope so they are not hardcoded.


In [0]:
import os
from openai import AzureOpenAI

# Load secrets from the Databricks-backed Key Vault scope and store them as env vars
scope = "aoai-scope"
os.environ["OPENAI_API_BASE"] = dbutils.secrets.get(scope, "openai-api-base")
os.environ["OPENAI_API_KEY"] = dbutils.secrets.get(scope, "openai-api-key")
os.environ["OPENAI_API_VERSION"] = dbutils.secrets.get(scope, "openai-api-version")
os.environ["OPENAI_DEPLOYMENT_NAME"] = dbutils.secrets.get(scope, "openai-deployment-name")

# Initialize the Azure OpenAI client once for reuse in this notebook
openai_client = AzureOpenAI(
    azure_endpoint=os.environ["OPENAI_API_BASE"],
    api_key=os.environ["OPENAI_API_KEY"],
    api_version=os.environ["OPENAI_API_VERSION"],
)

# Keep the deployment name handy for later calls
deployment_name = os.environ["OPENAI_DEPLOYMENT_NAME"]


## <span style="color:#1f77b4">**Chat Completions setup**</span>

Define the system and user prompts and send them to the Azure OpenAI deployment using the chat completions API.


### <span style="color:#1f77b4">**Define the system prompt**</span>

The system prompt sets the assistant behavior and tone for the response.


In [0]:
# System prompt sets the assistant behavior for all completions
system_prompt = f""" You are a helpful AI assistant meant to answer the user query """


### <span style="color:#1f77b4">**Define the user prompt**</span>

The user prompt is the query we want the assistant to answer.


In [0]:
# User prompt is the question we send to the model
user_prompt = f""" Hi how are you?"""


### <span style="color:#1f77b4">**Call the GPT deployment**</span>

Call the deployment and print the assistant response text.


In [0]:
# Call the chat completions endpoint with system + user messages
gpt_response = openai_client.chat.completions.create(
    model = deployment_name,
    messages = [
        {
            "role":"system", "content": f"{system_prompt}"
        },
        {
            "role":"user", "content": f"{user_prompt}"
        }
    ],
    temperature = 0.7
)

# Print the assistant response text
print(gpt_response.choices[0].message.content)


Hi there! I’m just a bunch of code, so I don’t have feelings, but I’m here and ready to help. 😊  
How are you doing today?


Trace(trace_id=tr-f5b01a1aaf8b5eb7b426100f275ddbf9)

## <span style="color:#1f77b4">**Wrap as an MLflow pyfunc**</span>

Create a PythonModel wrapper so the chat logic can be logged and served from MLflow.


In [0]:
import os
import mlflow
from mlflow import pyfunc
from openai import AzureOpenAI

# Wrap Azure OpenAI calls as an MLflow pyfunc so it can be served
class BasicChatBot(pyfunc.PythonModel):
      def _get_client(self):
          # Build a client from env vars injected by the serving endpoint
          return AzureOpenAI(
            azure_endpoint=os.environ["OPENAI_API_BASE"],
            api_key=os.environ["OPENAI_API_KEY"],
            api_version=os.environ["OPENAI_API_VERSION"],
          )

      def chatCompletionsAPI(self, user_query):
          # Create a single completion for the provided user query
          response = self._get_client().chat.completions.create(
            model = os.environ["OPENAI_DEPLOYMENT_NAME"],
            messages = [
                {
                    "role":"system",
                    "content":"You are a helpful AI assistant"
                },
                {
                    "role":"user",
                    "content":f"{user_query}"
                }
            ],
            temperature = 0.7
          )

          return response.choices[0].message.content

      def predict(self, context, data):
          # MLflow passes a pandas DataFrame; extract the user_query column
          user_query = data["user_query"].iloc[0]
          gpt_response = self.chatCompletionsAPI(user_query)
          return gpt_response


/databricks/python/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:155: FutureWarning: Model's `predict` method contains invalid parameters: {'data'}. Only the following parameter names are allowed: context, model_input, and params. Note that invalid parameters will no longer be permitted in future versions.
  param_names = _check_func_signature(func, "predict")
/databricks/python/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


## <span style="color:#1f77b4">**Save the model locally**</span>

Save the pyfunc model to a local artifact directory with a signature and input example.


In [0]:
# Instantiate the model wrapper for saving and testing
test_model = BasicChatBot()


In [0]:
from mlflow.models import infer_signature
import pandas as pd
import shutil
from pathlib import Path

# Provide an input example for signature inference
input_example = pd.DataFrame([{"user_query": "hello how are you?"}])
signature = infer_signature(input_example)
model_path = "basicchatbot"

# Remove existing directory to avoid save conflicts on reruns
model_dir = Path(model_path)
if model_dir.exists():
    shutil.rmtree(model_dir)

# Save the pyfunc model locally
mlflow.pyfunc.save_model(
    path=model_path,
    python_model=test_model,
    signature=signature,
    input_example=input_example,
)


2025/12/30 20:18:34 INFO mlflow.pyfunc: Validating input example against model signature


## <span style="color:#1f77b4">**Load the saved model**</span>

Reload the saved artifact to confirm it can be deserialized.


In [0]:
# Load our custom model from the local artifact store
loaded_pyfunc_model = mlflow.pyfunc.load_model(model_path)


## <span style="color:#1f77b4">**Test the saved model**</span>

Run a sample inference to validate the predict path.


In [0]:
# Run a simple test inference through the saved model
model_input = pd.DataFrame([{"user_query": "hello how are you?"}])

model_response = loaded_pyfunc_model.predict(model_input)

print(model_response)


Hello! I’m just a program, so I don’t have feelings, but I’m here and ready to help you. 😊  
How are you doing today?


Trace(trace_id=tr-37fd9cbd1a12ef481546047e530e7077)

## <span style="color:#1f77b4">**Register the model in MLflow**</span>

Log and register the model in the workspace registry, then print the latest version.


In [0]:
import os
import mlflow
from mlflow.tracking import MlflowClient

# Avoid recording env vars (secrets) in model metadata
os.environ["MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING"] = "false"

# Use the workspace registry (not Unity Catalog)
mlflow.set_registry_uri("databricks")

model_name = "basic-chatbot"

with mlflow.start_run() as run:
    # Log and register the model in one step
    mlflow.pyfunc.log_model(
        name="model",
        python_model=test_model,
        signature=signature,
        input_example=input_example,
        registered_model_name=model_name,
    )
    client = MlflowClient()
    # Find the latest registered version
    versions = client.search_model_versions(f"name='{model_name}'")
    version = max((int(v.version) for v in versions), default=None)
    print(f"Registered model: {model_name}, version: {version}")


🔗 View Logged Model at: https://adb-7405605517349658.18.azuredatabricks.net/ml/experiments/2548890231483242/models/m-f88e51486d9b40e5ade6d87602a95304?o=7405605517349658
2025/12/30 20:20:32 INFO mlflow.pyfunc: Validating input example against model signature
Registered model 'basic-chatbot' already exists. Creating a new version of this model...
2025/12/30 20:22:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: basic-chatbot, version 2
Created version '2' of model 'basic-chatbot'.


Registered model: basic-chatbot, version: 2


## <span style="color:#1f77b4">**Query the serving endpoint**</span>

Example payload you can send to the model serving endpoint.


In [0]:
# Example serving request payload
{
  "dataframe_split": {
    "columns": [
      "user_query"
    ],
    "data": [
      [
        "hello how are you?"
      ]
    ]
  }
}


{'dataframe_records': [{'user_query': 'Tell me something interesting about AI'}]}